In [1]:
import codecs
import string
import unicodedata
import numpy as np
import sklearn.feature_selection as fs
import gensim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Binarizer
from sklearn.pipeline import Pipeline
from sklearn import naive_bayes
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.cross_validation import KFold, LeaveOneOut, StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.dummy import DummyClassifier
from nltk import word_tokenize          
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import doc2vec

In [17]:
all_ne = "juri-all-non-empty.csv"
all_e = "juri.csv"
docs = "court_rulings_task2_8classes_train.csv"
docs2 = "court_rulings_task2_6classes_train.csv"


stop = stopwords.words('french')
stop.append(u'dun')
stop.append(u'dune')
stop.append(u'les')

In [7]:
class LemmaTokenizer(object):
    def __init__(self):
        self.stemmer = SnowballStemmer("french")

    def __call__(self, doc):
        return [self.stemmer.stem(t) for t in word_tokenize(doc)]


def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii


def get_preprocessor(suffix=''):
    def preprocess(unicode_text):
        return unicode_text.strip().lower() + suffix
    return preprocess

def preprocess_data(X, n, suffix='', binarize=True):
    vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1),
                                 preprocessor=get_preprocessor(suffix), tokenizer=LemmaTokenizer())
    X = vectorizer.fit_transform(X)
    X = Binarizer(copy=False).fit_transform(X) if binarize else X
    return X

In [8]:
def load_data(filename=all_e):
    index, text, rule, area, date, claw = [], [], [], [], [], []
    with codecs.open(filename, 'r', encoding="utf-8") as f:
        for line in f:
            a = line.split("\t")
            if (line != "") and (len(a)==7):
                idx, loc, dec, dt, dsc, art, law  = line.split("\t") 
                if (loc != "") and (dsc != "") and (dec!="") and (dt!="") and (law!=""):
                    index.append(idx)
                    area.append(loc)
                    rule.append(dec)
                    d = str(int(dt.split("-")[0])/10)
                    date.append(d)
                    text.append(dsc)
                    claw.append(law)
                
    print len(rule), len(text)
#     text = np.array(text)
#     rule = np.array(rule)
    
#     rule = reduce_classes(rule)
    return index, text, area, date, rule, claw

In [8]:
idx, X, y_a, y_d, y_r, y_l = load_data(all_ne)

81006 81006


In [27]:
# idx, X, y_a, y_d, y_r, y_l, = load_data(all_e)

0 0


In [10]:
def load_data2(fin):
    legal, y = [], []
    with codecs.open(fin, 'r', encoding="utf-8") as f:
        ct = f.read()
    ct = ct.split("\n")
    print "len ct", len(ct)
    for i in range(len(ct)):
        aux = ct[i].split("\t")
        if len(aux) == 2:
            dsc, dec = aux[0], aux[1]
            legal.append(remove_accents(dsc))
            y.append(remove_accents(dec))
        else:
            print i
    print len(y), len(legal)
    # legal = np.array(legal)
    y = np.array(y)
    set_y = list(set(y))

    for i in range(len(legal)):
        legal[i] = legal[i].lower()
        legal[i] = ((legal[i].encode("utf8")).translate(None, string.punctuation)).decode("utf8")
        legal[i] = ((legal[i].encode("utf8")).translate(None, "1234567890")).decode("utf8")
        legal[i] = legal[i].replace("annule", "")
        legal[i] = legal[i].replace("casse", "")
        legal[i] = legal[i].replace("rejeter", "")
        legal[i] = legal[i].replace("rejette", "")
        legal[i] = legal[i].replace("irrecevable", "")
        legal[i] = legal[i].replace("irrecevabilite", "")
        legal[i] = legal[i].replace("recevable", "")
        legal[i] = legal[i].replace("recevabilite", "") 
        legal[i] = legal[i].replace("question", "")
        legal[i] = legal[i].replace("prioritaire", "")
    for j in range(len(set_y)):
            legal[i] = legal[i].replace(set_y[j], "")
    # y = reduce_classes(set_y)
    print "reduced classes", len(set(set_y))
    return legal, y

In [19]:
X, y = load_data2(docs2)

len ct 63048
63047
63047 63047
reduced classes 6


In [29]:
y[6000:6300]

array(['cassation', 'cassation', 'cassation', 'rejet', 'cassation',
       'rejet', 'cassation', 'rejet', 'cassation', 'cassation', 'rejet',
       'rejet', 'rejet', 'cassation', 'rejet', 'rejet', 'cassation',
       'cassation', 'rejet', 'cassation', 'rejet', 'rejet', 'cassation',
       'cassation', 'rejet', 'rejet', 'cassation', 'rejet', 'rejet',
       'cassation', 'cassation', 'rejet', 'cassation', 'rejet', 'rejet',
       'rejet', 'rejet', 'rejet', 'cassation', 'rejet', 'rejet', 'rejet',
       'rejet', 'cassation', 'cassation', 'rejet', 'cassation', 'rejet',
       'rejet', 'cassation', 'cassation', 'cassation', 'rejet', 'rejet',
       'cassation', 'cassation', 'rejet', 'rejet', 'rejet', 'rejet',
       'rejet', 'rejet', 'rejet', 'rejet', 'cassation', 'cassation',
       'rejet', 'rejet', 'rejet', 'cassation', 'rejet', 'cassation',
       'cassation', 'rejet', 'rejet', 'rejet', 'cassation', 'cassation',
       'rejet', 'rejet', 'cassation', 'rejet', 'rejet', 'rejet',
       'ca

----------------
2.Preprocess

In [12]:
def text_clean_up(text, labels=[], mask=False, mode='ruling', strip=True):
    new_text = text
    for i in range(len(new_text)):
        aux = new_text[i]
        if strip == True:
            aux = remove_accents(aux)
        aux = aux.lower()
        aux = ((aux.encode("utf8")).translate(None, string.punctuation)).decode("utf8")
        aux = ' '.join(aux.split())
        aux = aux.replace("non lieu", "non-lieu")
        aux = aux.replace("nonlieu", "non-lieu")
        if mask == True:
            if mode == 'ruling':
                aux = aux.replace("annule", "")
                aux = aux.replace("annuler", "")
                aux = aux.replace("casse", "")
                aux = aux.replace("casser", "")
                aux = aux.replace("cassant", "")
                aux = aux.replace("rejeter", "")
                aux = aux.replace("rejette", "")
                aux = aux.replace("irrecevable", "")
                aux = aux.replace("irrecevabilite", "")
                aux = aux.replace("recevable", "")
                aux = aux.replace("recevabilite", "")
                aux = aux.replace("recu", "")
#                 if len(labels[i].split()) > 1:
#                     l = labels[i].split()
#                     for j in range(len(l)):
#                         aux = aux.replace(l[j], "")
#                 else:  
                for j in range(len(labels)):
                    aux = aux.replace(labels[j], "")
            elif mode == 'area':
                a = (((labels[i].lower().replace("_", " ")).encode("utf8")).translate(None, "1234567890")).decode("utf8")
                aux = aux.replace(a, "")
                a = a.split()
                for j in range(len(a)):
                    aux.replace(a[j], "")
            elif mode == 'temp':
                aux = ((aux.encode("utf8")).translate(None, "1234567890")).decode("utf8")
            else: 
                print "error: mode unimplemented"
                return     
        new_text[i] = aux        
    return new_text    

def preprocess_classes(y, mode='ruling', setup='mword'):
    print "unique labels:", len(set(y))
    new_y = []
    for i in range(len(y)):
        if mode =='ruling':
            aux = y[i]
            aux = aux.lower()
            aux = aux.replace("'", " ")
            aux = aux.replace("`", " ")
            aux = ((aux.encode("utf8")).translate(None, string.punctuation)).decode("utf8")   
            aux = ' '.join(aux.split())
            aux = aux.replace("nonlieu", "non-lieu")
            aux = aux.replace("non lieu", "non-lieu")
            if setup == '1word':
                aux = aux.split(" ")[0]
        elif mode == 'area':
            aux = y[i]
        elif mode == 'temp1':
            aux = y[i]
            if int(aux) < 196:
                aux = '<196'
        elif mode == 'temp2':
            aux = y[i]
            if int(aux) <= 184:
                aux = '<=184'
            elif int(aux) > 184 and int(aux) <=186:
                aux = '185-186'
            elif int(aux) > 186 and int(aux) <=188:
                aux = '187-188'
            elif int(aux) > 188 and int(aux) <=191:
                aux = '189-191'
            else:
                aux = aux
        elif mode == 'none':
            aux = y[i]
        else:
            print "error: mode unimplemented"
            return 
        new_y.append(aux)
    print "unique labels after initial preprocessing: ", len(set(new_y))
    return new_y

In [30]:
X = text_clean_up(X,list(set(y)), True)

In [33]:
def load_data_xy(fname):
    text, label = [], []
    with codecs.open(fname, 'r', encoding="utf-8") as f:
        for line in f:
            a = line.split("\t")
            if (line != "") and (len(a)==2):
                text.append(a[0])
                label.append(a[1])
#     text, label = np.array(text), np.array(label)
    return text, label

In [34]:
new_yr = preprocess_classes(y_r, 'ruling')
new_Xr = text_clean_up(X, new_yr, True, 'ruling')
new_yr2 = preprocess_classes(y_r, 'ruling', '1word')
new_Xr2 = text_clean_up(X, new_yr2, True, 'ruling')

new_ya = preprocess_classes(y_a, 'area')
new_Xa = text_clean_up(X, new_ya, True, 'area')

new_yd = preprocess_classes(y_d, 'temp1')
new_Xd = text_clean_up(X, new_yd, True, 'temp')
new_yd2 = preprocess_classes(y_d, 'temp2')
new_Xd2 = text_clean_up(X, new_yd2, True, 'temp')

unique labels: 436
unique labels after initial preprocessing:  425
unique labels: 436
unique labels after initial preprocessing:  48
unique labels: 17
unique labels after initial preprocessing:  17
unique labels: 21
unique labels after initial preprocessing:  7
unique labels: 21
unique labels after initial preprocessing:  14


1.3 Reduce Classes

In [35]:
def get_label_counts(y):
    '''get number of entries (court rulings) 
    for each label'''
    y_s = list(set(y))
    idx = np.zeros(len(y_s), dtype=int)
    for i in range(len(y)):
        for j in range(len(y_s)):
            if y[i] == y_s[j]:
                idx[j] += 1
    return y_s, idx

def get_frequent_labels(y, y_rs, idx, threshold=0):
    '''get labels that have the number of entries 
    above threshold'''
    count = 0
    y_rsr = []
    mask_y = np.zeros(len(y), dtype=int)
    for i in range(len(idx)):
        if idx[i] > threshold:
            print y_rs[i], "&", idx[i]
            y_rsr.append(y_rs[i])
            count += idx[i]
    print "number of labels with total entries above ", threshold, ": ", count
    
    for i in range(len(y)):
        if y[i] in y_rsr:
            mask_y[i] = 1   
    return mask_y

def reduce_classes(X, y, threshold=0):
    new_y, new_X = [], []
    print "initial classes: ", len(set(y))
    y_s, idx = get_label_counts(y)
    mask = get_frequent_labels(y, y_s, idx, threshold)
    for i in range(len(mask)):
        if mask[i] == 1:
            new_y.append(y[i])
            new_X.append(X[i])
    print "classes after reduction: ", len(set(new_y))
    return new_X, new_y

In [36]:
new_Xr, new_yr = reduce_classes(new_Xr, new_yr, 200)
new_Xr2, new_yr2 = reduce_classes(new_Xr2, new_yr2, 200)
new_Xa, new_ya = reduce_classes(new_Xa, new_ya, 200)
new_Xd, new_yd = reduce_classes(new_Xd, new_yd)
new_Xd2, new_yd2 = reduce_classes(new_Xd2, new_yd2, 1000)

initial classes:  425
cassation partielle & 6771
cassation partielle sans renvoi & 869
cassation partielle cassation & 960
cassation & 27375
irrecevabilité & 1585
cassation partielle rejet cassation & 798
cassation sans renvoi & 1804
rejet & 37750
number of labels with total entries above  200 :  77912
classes after reduction:  8
initial classes:  48
annulation & 305
cassation & 39842
rejet & 38209
irrecevabilité & 1872
number of labels with total entries above  200 :  80228
classes after reduction:  4
initial classes:  17
CHAMBRE_CIVILE_1 & 12194
CHAMBRE_CIVILE_2 & 10321
CHAMBRE_CIVILE_3 & 10802
CHAMBRE_SOCIALE & 20397
CHAMBRE_CRIMINELLE & 15317
CHAMBRE_COMMERCIALE & 11052
ASSEMBLEE_PLENIERE & 391
number of labels with total entries above  200 :  80474
classes after reduction:  7
initial classes:  7
201 & 4541
200 & 12577
199 & 16693
198 & 18233
197 & 23964
196 & 4797
<196 & 201
number of labels with total entries above  0 :  81006
classes after reduction:  7
initial classes:  14
201 

In [27]:
trainr2_index, testr2_index = next(iter(StratifiedKFold(np.array(new_yr2), 2)))

In [28]:
trainr_index, testr_index = next(iter(StratifiedKFold(np.array(new_yr), 2)))
traina_index, testa_index = next(iter(StratifiedKFold(np.array(new_ya), 2)))
traind_index, testd_index = next(iter(StratifiedKFold(np.array(new_yd), 2)))

In [ ]:
[], [], [], [], trainr_index, testr_index = train_test_split(np.zeros(len(new_Xr)), np.array(new_yr), 
                                                                               range(len(new_Xr)), test_size=0.30, random_state=42)
[], [], [], [], trainr_index, testr_index = train_test_split(np.zeros(len(new_Xr)), np.array(new_ya), 
                                                                               range(len(new_Xa)), test_size=0.30, random_state=42)
[], [], [], [] , trainr_index, testr_index = train_test_split(np.zeros(len(new_Xr)), np.array(new_yd), 
                                                                               range(len(new_Xd)), test_size=0.30, random_state=42)


In [29]:
def get_fancy_index(x, y, idx):
    x_t, y_t = [], []
    print "entered get_fancy_indexing"
    for i in range(len(idx)):
        x_t.append(x[idx[i]])
        if type(y[idx[i]]) == int:
            print i
        y_t.append(y[idx[i]])
    return x_t, y_t

def write_train_test(ftr, fts, x, y, train, test):  
    print "entered write_train_test"
    x_train, y_train = get_fancy_index(x, y, train)
    with codecs.open(ftr, "w", encoding="utf-8") as f:
        for i in range(len(x_train)):
            f.write(x_train[i] +"\t" + y_train[i] + "\n")

    x_test , y_test = get_fancy_index(x, y, test)
    with codecs.open(fts, "w", encoding="utf-8") as f:
        for i in range(len(x_test)):
            f.write(x_test[i] +"\t" + y_test[i] + "\n")
    

In [30]:
write_train_test("french_court_rulings_task2_8classes_train.csv", "court_rulings_task2_8classes_test.csv", 
                 new_Xr2, new_yr2, trainr2_index, testr2_index)

entered write_train_test
entered get_fancy_indexing
entered get_fancy_indexing


In [31]:
write_train_test("french_court_rulings_task2_6classes_train.csv", "court_rulings_task2_6classes_test.csv", 
                 new_Xr, new_yr, trainr_index, testr_index)
write_train_test("french_court_rulings_task1_8classes_train.csv", "court_rulings_task1_8classes_test.csv", 
                 new_Xa, new_ya, traina_index, testa_index)
write_train_test("french_court_rulings_task3_7classes_train.csv", "court_rulings_task3_7classes_test.csv", 
                 new_Xd, new_yd, traind_index, testd_index)


entered write_train_test
entered get_fancy_indexing
entered get_fancy_indexing
entered write_train_test
entered get_fancy_indexing
entered get_fancy_indexing
entered write_train_test
entered get_fancy_indexing
entered get_fancy_indexing


--------------------------
4.Feature Pipe

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import clone

In [32]:
def get_best_features(X, y, vectorizer, n):
    '''get names of best features in X from vectorizer'''
    print "entered get_best_features"
    f = codecs.open("best_k.txt", "a", encoding="utf-8")
    fnames = vectorizer.get_feature_names()
    b = fs.SelectKBest(fs.f_classif, k=50) #k is number of features.
    X_n = b.fit_transform(X, y)
    index_v =  b.get_support()

    f.write("best "+ n +"grams:\n")
    for i in range(len(index_v)):
        if index_v[i] == True:
            f.write(fnames[i])
            f.write("\n")
    f.close()
    print "exited get_best_features"

In [33]:
vectorizer_un = CountVectorizer(analyzer='word', ngram_range=(1, 1))
vectorizer_bi = CountVectorizer(analyzer='word', ngram_range=(2, 2))
vectorizer_tri = CountVectorizer(analyzer='word', ngram_range=(3, 3))

In [34]:
# Xr_new = CountVectorizer(analyzer='word', ngram_range=(2, 2)).fit_transform(new_Xr)
# Xr_new2 = CountVectorizer(analyzer='word', ngram_range=(2, 2)).fit_transform(new_Xr2)
# Xa_new = CountVectorizer(analyzer='word', ngram_range=(2, 2)).fit_transform(new_Xa)
# Xd_new = CountVectorizer(analyzer='word', ngram_range=(2, 2)).fit_transform(new_Xd)
# Xd_new2 = CountVectorizer(analyzer='word', ngram_range=(2, 2)).fit_transform(new_Xd2)
X_new_un = vectorizer_un.fit_transform(X)
X_new_bi = vectorizer_bi.fit_transform(X)
X_new_tri= vectorizer_tri.fit_transform(X)

In [35]:
get_best_features(X_new_un, y, vectorizer_un, "1")
get_best_features(X_new_bi, y, vectorizer_bi, "2")
get_best_features(X_new_tri, y, vectorizer_tri, "3")

entered get_best_features
exited get_best_features
entered get_best_features
exited get_best_features
entered get_best_features
exited get_best_features


In [35]:
def calculate_lexical_density(x):
    '''unique tokens/ tokens''' 
    print "entered calculate_lexical_density"
    print "len x", len(x)
    new_x = []
    for i in range(len(x)):
        unique_w = len(set(x[i].split()))
        new_x.append(float(unique_w)/float(len(x[i])))   
    print "type x", type(new_x)
    print "len new_x", len(new_x)
    new_x = np.array(new_x)
    return new_x[:, np.newaxis]
    
def calculate_lexical_richness(x):
    '''unique stems / stems'''
    print "entered calculate_lexical_richness"
    stemmer = SnowballStemmer("french")
    new_x = []
    for i in range(len(x)):
        stems = []
        for word in x[i].split():
            stems.append(stemmer.stem(word))         
        unique_s = len(set(stems))
        new_x.append(float(unique_s)/float(len(stems)))
    
    print "len new_x:", len(new_x)
    new_x = np.array(new_x)
    return new_x[:,np.newaxis]


In [36]:
from sklearn.preprocessing import FunctionTransformer

In [37]:
ttr_getter = FunctionTransformer(calculate_lexical_density, validate=False)
str_getter = FunctionTransformer(calculate_lexical_richness, validate=False)

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import clone
from imblearn.over_sampling import SMOTE

In [39]:
feature_union = FeatureUnion([
                            ('type-token-ratio',ttr_getter),
#                              ('stem-token-ratio', str_getter),
                              ('bow', CountVectorizer(analyzer='word', ngram_range=(2,2)))
                             ])



clf_pipe = Pipeline([('features', feature_union),
                     ('svm',LinearSVC(C=0.1, class_weight='balanced'))
                    ])

In [328]:
sm = SMOTE(random_state=42)
# X_res, y_res = sm.fit_sample(new_Xd, new_yd)
# X_res2, y_res2 = sm.fit_sample(new_Xd2, new_yd2)

In [308]:
Xd_new = feature_union.fit_transform(new_Xd)
Xd_new2 = feature_union.fit_transform(new_Xd2)

entered calculate_lexical_density
len x 126865
type x <type 'list'>
len new_x 126865


-------------------------------------

Doc to vec

In [42]:
# model = doc2vec.Doc2Vec.load("court-task1-8_test.d2v2")
model2_tr = doc2vec.Doc2Vec.load("court-task2-8_train.d2v2")
# model2_ts = doc2vec.Doc2Vec.load("court-task2-8_test.d2v2")

In [43]:
def get_docvecs_np(vecs):
    np_vecs = np.zeros((len(vecs), len(vecs[0])), dtype=vecs[0].dtype)
    for i in range(len(vecs)):
        np_vecs[i] = vecs[i]
    print "types", type(np_vecs)
    return np_vecs

In [44]:
X_new_d2v = get_docvecs_np(model2_tr.docvecs)

types <type 'numpy.ndarray'>


In [48]:
len(X_new_d2v)

61358

---------------------
5.Cross Validation

In [50]:
def cross_val(X, y, clf, dummy, cv, mode='np'):
    accuracy, recall, precision, f1 = [], [], [], []
    dummy_acc, dummy_rec, dummy_prec, dummy_f1 = [], [], [], []
    print "-------------"
    print "entered cross_val"
    
    for train_index, test_index in cv:
        if mode == 'list':
            X_train, y_train = get_fancy_index(X, y, train_index)
            X_test, y_test = get_fancy_index(X, y, test_index)
            y_train, y_test = np.array(y_train), np.array(y_test)
        
        elif mode == 'np':   
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
        else:
            print "error: unimplemented mode"
            
        print "fitting the classifier"
        clf.fit(X_train, y_train)
        dummy.fit(X_train, y_train)

        print "predicting"
        y_pred = clf.predict(X_test)
        y_dummy = dummy.predict(X_test)
        
        print "type test pred", type(y_test), type(y_pred), len(y_test), len(y_pred), y_pred.shape, y_test.shape

        print "svm report:\n", classification_report(y_test, y_pred)
        print "dummy report: \n", classification_report(y_test, y_dummy)

        accuracy.append(accuracy_score(y_test, y_pred))
        precision.append(precision_score(y_test, y_pred, average='weighted'))
        recall.append(recall_score(y_test, y_pred, average='weighted'))
        f1.append(f1_score(y_test, y_pred, average='weighted'))

        dummy_acc.append(accuracy_score(y_test, y_dummy))
        dummy_prec.append(precision_score(y_test, y_dummy, average='weighted'))
        dummy_rec.append(recall_score(y_test, y_dummy, average='weighted'))
        dummy_f1.append(f1_score(y_test, y_dummy, average='weighted'))
        print "accuracy mean ", np.mean(accuracy), " accuracy std ", np.std(accuracy)
        
    print "precision mean ", np.mean(precision), " and std ", np.std(precision)
    print "recall mean ", np.mean(recall), " and std ", np.std(recall)
    print "f1 mean ", np.mean(f1), " and std ", np.std(f1)
    print "dummy accuracy mean ", np.mean(dummy_acc), " accuracy std ", np.std(dummy_acc)
    print "dummy precision mean ", np.mean(dummy_rec), " and std ", np.std(dummy_prec)
    print "dummy recall mean ", np.mean(dummy_rec), " and std ", np.std(dummy_rec)
    print "dummy f1 mean ", np.mean(dummy_f1), " and std ", np.std(dummy_f1)
    print "------------"
    print "exited cross_val"
        

In [58]:
# skf_r = StratifiedKFold(new_yr, n_folds=10)
# skf_r2 = StratifiedKFold(new_yr2, n_folds=10)
# skf_a = StratifiedKFold(new_ya, n_folds=10)
# skf_d = StratifiedKFold(new_yd, n_folds=10)
# skf_d2 = StratifiedKFold(new_yd2, n_folds=10)
skf_d2v = StratifiedKFold(y, n_folds=10)
# clf = LinearSVC(C=0.01, class_weight='balanced')
clf = KNeighborsClassifier(n_neighbors=8)
dummy = DummyClassifier(strategy="stratified")

In [59]:
cross_val(X_new_d2v, y, clf, dummy, skf_d2v, 'np')

-------------
entered cross_val
fitting the classifier
predicting
type test pred <type 'numpy.ndarray'> <type 'numpy.ndarray'> 6139 6139 (6139L,) (6139L,)
svm report:
             precision    recall  f1-score   support

  cassation       0.30      0.56      0.39      1883
cassation partielle       0.05      0.00      0.00       478
cassation partielle cassation       1.00      0.03      0.07        59
cassation partielle rejet cassation       0.00      0.00      0.00        46
cassation partielle sans renvoi       0.00      0.00      0.00        51
cassation sans renvoi       0.00      0.00      0.00       104
irrecevabilite       0.07      0.50      0.12       119
      rejet       0.54      0.27      0.36      3399

avg / total       0.41      0.33      0.32      6139

dummy report: 
             precision    recall  f1-score   support

  cassation       0.31      0.30      0.30      1883
cassation partielle       0.09      0.09      0.09       478
cassation partielle cassation     

Begining of Minidemo
----------------------------------------------

In [44]:
X_train, y_train  = load_data_xy("court_rulings_task2_6classes_train.csv")
X_test, y_test = load_data_xy("court_rulings_task2_6classes_test.csv")

In [52]:
X = X_train + X_test

In [54]:
X_new = CountVectorizer(analyzer='word', ngram_range=(2, 2)).fit_transform(X)

In [55]:
X_train_new = X_new[:len(X_train)]
X_test_new = X_new[len(X_train):]

In [56]:
clf.fit(X_train_new, y_train)
y_pred = clf.predict(X_test_new)

In [58]:
y_pred[30], y_test[30], X[30]

(u'cassation\n',
 u'cassation\n',
 u'sur le moyen unique attendu que dame z reproche a l arret confirmatif attaque paris  novembre  de l avoir declaree personnellement en liquidation des biens par application de l article  de la loi du  juillet  alors selon le pourvoi que les simples actes de gestion et d administration ainsi que la seule qualite de dirigeant de fait de la societe quot internationale d application des techniques electriques mecaniques pneumatiques et connexes quot sinatnem ne suffisent pas a justifier l extension a dame michel de y des biens de cette societe qu en l espece la cour d appel n a pas releve un acte quelconque effectue par le dirigeant dans son interet personnel ni aucun fait precis revelant qu il avait dispose des biens sociaux comme des siens propres et qu elle n a pas plus indique les circonstances de fait susceptibles de faire apparaitre que la poursuite de l exploitation etait abusive ou dictee par le seul interet du dirigeant qu en definitive la cour 

End of Minidemo
--------------------------------------

In [ ]:
new_yd2[0]

In [327]:
# cross_val(Xr_new, np.array(new_yr), clf, dummy, skf_r)
# cross_val(Xr_new2, np.array(new_yr2), clf, dummy, skf_r2)
# cross_val(Xa_new, np.array(new_ya), clf, dummy, skf_a)
# cross_val(Xd_new, np.array(new_yd), clf, dummy, skf_d)
cross_val(Xd_new2, np.array(new_yd2), clf, dummy, skf_d2)
# cross_val(new_Xd, np.array(new_yd), clf_pipe, dummy, skf_d, mode='list')
# cross_val(new_Xd2, np.array(new_yd2), clf_pipe, dummy, skf_d2, mode='list')

-------------
entered cross_val
fitting the classifier
predicting
type test pred <type 'numpy.ndarray'> <type 'numpy.ndarray'> 12659 12659 (12659L,) (12659L,)
svm report:
             precision    recall  f1-score   support

        196       0.76      0.98      0.86      3243
        197       0.54      0.62      0.58      2675
        198       0.50      0.38      0.43      2363
        199       0.71      0.59      0.64      2067
        200       0.82      0.74      0.78      1679
        201       1.00      0.56      0.72       632

avg / total       0.68      0.68      0.67     12659

dummy report: 
             precision    recall  f1-score   support

        196       0.26      0.26      0.26      3243
        197       0.21      0.21      0.21      2675
        198       0.18      0.18      0.18      2363
        199       0.17      0.17      0.17      2067
        200       0.12      0.12      0.12      1679
        201       0.04      0.04      0.04       632

avg / total   

In [135]:
from collections import Counter
from sklearn.datasets import fetch_mldata
from imblearn.over_sampling import SMOTE

pima = fetch_mldata('diabetes_scale')
X, y = pima['data'], pima['target']
print('Original dataset shape {}'.format(Counter(y)))

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(X, y)
print('Resampled dataset shape {}'.format(Counter(y_res)))


Original dataset shape Counter({1: 500, -1: 268})
Resampled dataset shape Counter({1: 500, -1: 500})


-------------------------
7.Get best features

In [ ]:
def get_best_features(X, y, vectorizer):
    '''get names of best features in X from vectorizer'''
    print "entered get_best_features"
    f = codecs.open("results.txt", "w", encoding="utf-8")
    fnames = vectorizer.get_feature_names()
    b = fs.SelectKBest(fs.f_classif, k=50) #k is number of features.
    X_n = b.fit_transform(X, y)
    index_v =  b.get_support()

    print "best unigrams:"
    for i in range(len(index_v)):
        if index_v[i] == True:
            f.write(fnames[i])
            f.write("\n")
    f.close()
    print "exited get_best_features"

---------------------------
8.Significance testing

In [30]:
# Author:  Alexandre Gramfort <alexandre.gramfort@inria.fr>
# License: BSD 3 clause

print(__doc__)

import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold, permutation_test_score
from sklearn import datasets


svm = LinearSVC(C=0.1)
cv = StratifiedKFold(new_yr, 10)

def get_significance(clf, X, y, cv):
    score, permutation_scores, pvalue = permutation_test_score(
        svm, X, y, scoring="f1", cv=cv, n_permutations=100, n_jobs=1)

    print("Classification score %s (pvalue : %s)" % (score, pvalue))
    return score, permutation_scores

score, permutation_scores, pvalue = get_significance(svm, new_Xr, np.array(new_yr), cv)

Automatically created module for IPython interactive environment


MemoryError: 

In [ ]:
score, permutation_scores, pvalue = get_significance(svm, new_Xr, new_yr, cv)
n_classes = len(set(new_yr))

In [ ]:
# View histogram of permutation scores
plt.hist(permutation_scores, 20, label='Permutation scores')
ylim = plt.ylim()
# BUG: vlines(..., linestyle='--') fails on older versions of matplotlib
#plt.vlines(score, ylim[0], ylim[1], linestyle='--',
#          color='g', linewidth=3, label='Classification Score'
#          ' (pvalue %s)' % pvalue)
#plt.vlines(1.0 / n_classes, ylim[0], ylim[1], linestyle='--',
#          color='k', linewidth=3, label='Luck')
plt.plot(2 * [score], ylim, '--g', linewidth=3,
         label='Classification Score'
         ' (pvalue %s)' % pvalue)
plt.plot(2 * [1. / n_classes], ylim, '--k', linewidth=3, label='Luck')

plt.ylim(ylim)
plt.legend()
plt.xlabel('Score')
plt.show()

2.2Get True Ruling

In [269]:
def get_rule_dct(y):
    '''get vocabulary of rulings 
    from ruling labels'''
    y_set = list(set(y))
    print "unique ruling labels", len(y_set)
    dct = []
    for i in range(len(y_set)):
        aux = y_set[i].split()
        for j in range(len(aux)):
            if aux[j] not in dct and aux[j] not in stop:
                dct.append(aux[j])
    print "unique words in ruling labels", len(set(dct))
    return dct

In [270]:
r_dct = get_rule_dct(new_yr)

unique ruling labels 414
unique words in ruling labels 142


In [462]:
# r_dct

In [261]:
def get_label_from_text(X, dct, split_char=' '):
    '''attempt to get label from text by 
    matching label with first word of the text'''
    new_y = []
    for i in range(len(X)):
        aux = X[i].split(split_char)[0]
#         print aux
        if aux in dct:
            new_y.append(aux)
    return new_y

In [262]:
new_y = split_text(X, r_dct, ' ')

In [263]:
new_y_set = list(set(new_y))

In [264]:
len(new_y), len(new_y_set), len(X)

(10819, 28, 81006)

In [463]:
# new_y_set

-------------------------
Train test split. 

In [24]:
train_index[0], yl[train_index[0]]

(70, u'CHAMBRE_CIVILE')

In [14]:
# tokens = []
with open("court_rulings_task2_8classes_train.csv") as f:
    for line in f:
        tokens = (gensim.utils.to_unicode(line).split())

In [15]:
tokens

[u'la',
 u'cour',
 u'de',
 u'cassation',
 u'a',
 u'rendu',
 u'l',
 u'arret',
 u'suivant',
 u'br',
 u'clear',
 u'none',
 u'br',
 u'clear',
 u'none',
 u'vu',
 u'la',
 u'connexite',
 u'joint',
 u'les',
 u'pourvois',
 u'n',
 u't',
 u'z',
 u'a',
 u'et',
 u'b',
 u'br',
 u'clear',
 u'none',
 u'br',
 u'clear',
 u'none',
 u'attendu',
 u'selon',
 u'les',
 u'arrets',
 u'attaques',
 u'aix',
 u'en',
 u'provence',
 u'septembre',
 u'que',
 u'm',
 u'x',
 u'm',
 u'y',
 u'm',
 u'z',
 u'et',
 u'm',
 u'a',
 u'ont',
 u'ete',
 u'engages',
 u'avant',
 u'le',
 u'er',
 u'juin',
 u'par',
 u'la',
 u'societe',
 u'carrefour',
 u'france',
 u'aux',
 u'droits',
 u'de',
 u'laquelle',
 u'se',
 u'trouve',
 u'la',
 u'societe',
 u'carrefour',
 u'hypermaches',
 u'en',
 u'qualite',
 u'de',
 u'vendeurs',
 u'qualifies',
 u'des',
 u'rayons',
 u'informatique',
 u'electromenager',
 u'gros',
 u'et',
 u'petit',
 u'photographie',
 u'cinema',
 u'et',
 u'son',
 u'soit',
 u'vendeurs',
 u'epcs',
 u'qu',
 u'un',
 u'accord',
 u'd',
 u'en